# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.kinetics_bounded import kinetic_block_rule
from physics.energy_bounded import energy_block_rule
from physics.VLE_bounded import VLE_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals,initialize=3000) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

# MPCC construction
model.beta = pe.Var(within=pe.NonNegativeReals,initialize=1)
model.s_L= pe.Var(within=pe.NonNegativeReals,initialize=0)
model.s_V= pe.Var(within=pe.NonNegativeReals,initialize=0)

# MPCC Parameters

In [5]:
model.epi = pe.Param(initialize=1e-3,mutable=True)

# Construct Individual Blocks

In [6]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [7]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [8]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [9]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [10]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.beta*model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

## MPCC: Beta

In [11]:
def beta_rule(model):
    return model.beta == 1-model.s_L + model.s_V
model.beta_con = pe.Constraint(rule=beta_rule)

def s_L_complementarity_rule(model):
    return (sum(model.L[s] for s in model.outlet) + model.s_L) <= ((sum(model.L[s] for s in model.outlet) - model.s_L)**2+model.epi)**0.5
model.s_L_complementarity_con = pe.Constraint(rule=s_L_complementarity_rule)

def s_V_complementarity_rule(model):
    return (sum(model.V[s] for s in model.outlet) + model.s_V) <= ((sum(model.V[s] for s in model.outlet) - model.s_V)**2+model.epi)**0.5
model.s_V_complementarity_con = pe.Constraint(rule=s_V_complementarity_rule)

## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [12]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [13]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [14]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [15]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [16]:
model.cat.fix(3000)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(1)
model.z['CO'].fix(1/(1+0.8)-0/2)
model.z['H2'].fix(0.8/(1+0.8)-0/2)
model.z['C30H62'].fix(0)
model.VLE_block.n_ave.fix(20)

In [17]:
check_DOF(pe,model)

Active Equality Constraints:	 1182
Active Inequality Constraints:	 2
Active Variables:		 1357
Fixed Variables:		 172
DOF:				 3


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [18]:
model.T.setub(300+273.15)

In [19]:
# model.beta.fix(1)
# model.s_L.fix(0)
# model.s_L.fix(0)

In [20]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.cat,sense=pe.minimize)
# model.obj = pe.Objective(expr = model.T ,sense=pe.maximize)
model.obj = pe.Objective(expr = model.T - 100000*sum(model.L[s] for s in model.outlet)*model.s_L ,sense=pe.maximize)

# epi = 1e-4

In [21]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
opt.options['linear_solver'] = 'ma97'
# opt.options['nlp_scaling_method'] = None
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma97
max_iter=7000


List of user-set options:

                                    Name   Value                used
                           linear_solver = ma97                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver ma97.

Number of nonzeros in equality constraint Jacobian...:     4012
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.....

  93 -5.2494504e+02 8.89e-02 1.07e+05  -5.7 1.65e+02    -  6.65e-01 1.45e-01h  1
  94 -5.2614582e+02 6.15e-01 2.34e+05  -5.7 1.95e+02    -  5.67e-01 1.00e+00f  1
  95 -5.2654519e+02 6.10e-01 3.02e+05  -5.7 5.54e+02    -  6.51e-01 1.18e-01f  1
  96 -5.2936546e+02 3.61e+00 1.15e+06  -5.7 6.75e+02    -  7.47e-01 6.84e-01f  1
  97 -5.3037921e+02 3.81e+00 1.28e+06  -5.7 2.50e+03    -  3.67e-01 6.73e-02f  1
  98 -5.3039294e+02 3.80e+00 1.20e+06  -5.7 1.47e+03    -  4.99e-06 2.06e-03h  1
  99 -5.3040806e+02 3.79e+00 2.78e+06  -5.7 1.30e+03    -  1.34e-06 3.24e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100 -5.3040949e+02 3.79e+00 2.61e+06  -5.7 1.10e+03    -  6.45e-07 7.47e-04h  1
 101 -5.3041104e+02 3.79e+00 2.61e+06  -5.7 3.73e+03    -  1.43e-05 6.72e-05h  1
 102 -5.3041104e+02 3.79e+00 2.61e+06  -5.7 3.05e+03    -  2.32e-10 2.34e-09f  2
 103r-5.3041104e+02 3.79e+00 1.00e+03   0.5 0.00e+00   4.9 0.00e+00 7.69e-12R  2
 104r-5.3037610e+02 3.79e+00

 188 -5.5016174e+02 1.49e-10 3.69e+05  -5.7 2.63e-05   5.7 9.79e-01 1.00e+00h  1
 189 -5.5018973e+02 1.39e-05 3.63e+05  -5.7 7.19e+02    -  8.69e-03 1.42e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190 -5.5243417e+02 9.12e-02 1.56e+08  -5.7 6.88e+02    -  1.86e-01 1.20e-01f  1
 191 -5.5243418e+02 9.12e-02 1.56e+08  -5.7 2.60e+02    -  1.78e-01 2.88e-06h  2
 192 -5.5363948e+02 6.73e-01 1.83e+10  -5.7 2.25e+02    -  6.37e-01 1.00e+00f  1
 193 -5.5484433e+02 6.75e-01 1.89e+10  -5.7 2.27e+02    -  1.00e+00 1.00e+00f  1
 194 -5.5484358e+02 5.53e-01 1.55e+10  -5.7 3.67e-02   5.2 3.29e-03 1.81e-01h  1
 195 -5.5484734e+02 1.46e-04 6.60e+08  -5.7 3.89e+00    -  3.62e-01 1.00e+00f  1
 196 -5.5485032e+02 2.99e-10 8.54e+04  -5.7 5.18e-05   5.6 1.00e+00 1.00e+00h  1
 197 -5.5485386e+02 4.88e-11 9.32e+03  -5.7 1.30e-04   5.1 1.00e+00 1.00e+00f  1
 198 -5.5486971e+02 3.25e-05 3.96e+05  -5.7 1.51e+02    -  1.45e-02 1.05e-02f  1
 199 -5.5553085e+02 2.05e-01

 264r-4.8105622e+02 2.97e+00 3.50e+03   0.4 4.86e-06   8.9 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 265r-4.8104708e+02 2.97e+00 3.50e+03   0.4 1.82e-06   9.3 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 266r-4.8101939e+02 2.97e+00 3.50e+03   0.4 5.46e-06   8.8 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 267r-4.8100889e+02 2.97e+00 3.50e+03   0.4 2.05e-06   9.2 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag 

 299r-4.8058973e+02 2.97e+00 3.48e+03   0.4 1.68e-06   9.3 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 300r-4.8056447e+02 2.97e+00 3.48e+03   0.4 5.03e-06   8.8 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 301r-4.8055500e+02 2.97e+00 3.48e+03   0.4 1.88e-06   9.3 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 302r-4.8052663e+02 2.97e+00 3.48e+03   0.4 5.65e-06   8.8 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error fla

...

 546r-4.7777472e+02 2.97e+00 3.36e+03   0.4 4.54e-06   8.9 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 547r-4.7776689e+02 2.97e+00 3.36e+03   0.4 1.70e-06   9.3 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 548r-4.7774345e+02 2.97e+00 3.36e+03   0.4 5.10e-06   8.8 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 549r-4.7773466e+02 2.97e+00 3.36e+03   0.4 1.91e-06   9.2 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag 

 582r-4.7740471e+02 2.97e+00 3.35e+03   0.4 4.70e-06   8.9 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 583r-4.7739670e+02 2.97e+00 3.35e+03   0.4 1.76e-06   9.3 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 584r-4.7739369e+02 2.97e+00 3.35e+03   0.4 6.61e-07   9.7 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 585r-4.7738467e+02 2.97e+00 3.35e+03   0.4 1.98e-06   9.2 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag 

 618r-4.7706631e+02 2.97e+00 3.34e+03   0.4 4.88e-06   8.8 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 619r-4.7705808e+02 2.97e+00 3.34e+03   0.4 1.83e-06   9.3 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620r-4.7705500e+02 2.97e+00 3.34e+03   0.4 6.86e-07   9.7 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 621r-4.7704575e+02 2.97e+00 3.33e+03   0.4 

 654r-4.7673885e+02 2.97e+00 3.32e+03   0.4 6.33e-07   9.7 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 655r-4.7673040e+02 2.97e+00 3.32e+03   0.4 1.90e-06   9.2 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 656r-4.7672723e+02 2.97e+00 3.32e+03   0.4 7.12e-07   9.7 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular
 657r-4.7671773e+02 2.97e+00 3.32e+03   0.4 2.13e-06   9.2 1.00e+00 1.00e+00f  1

 Error return from ma97_factor. Error flag =  -7
 Matrix found to be singular

 Error return from ma97_factor. Error flag 

ApplicationError: Solver (ipopt) did not exit normally

In [ ]:
model.Q_main.value

In [ ]:
model.T.value

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reactive_flash_300C_n58.pickle','wb') as f:
#     pickle.dump(results,f)

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}'.format(model.x[i].value,model.y[i].value))

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = []; rf_data['s_L'] = []; rf_data['s_V'] = []; rf_data['beta'] = [];

for i in model.r_total_comp:
    rf_data['r'][i] = []
    
Trange = np.arange(300+273.15,199+273.15,-2)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {0:.2f} K|\tV = {1:.5f}|\tL = {2:.5f}|\ts_L = {3:.2f}|\tbeta={4:.2f}'.format(model.T.value,model.V['out'].value,model.L['out'].value,model.s_L.value,model.beta.value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    

    for i in model.r_total_comp:
        rf_data['r'][i].append(model.r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    rf_data['r_WGS'].append(model.kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.kinetics_block.r_FT_total.value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)
    rf_data['s_L'].append(model.s_L.value)
    rf_data['s_V'].append(model.s_V.value)
    rf_data['beta'].append(model.beta.value)

# Optimize across disappearing phases

In [ ]:
model.T.fixed = True
model.cat.fixed = False
model.cat.setlb(1)

In [ ]:
check_DOF(pe,model)

In [ ]:
model.del_component(model.obj)
model.obj = pe.Objective(expr = model.cat, sense=pe.minimize)

In [ ]:
results = opt.solve(model,tee=True)
update_dual(pe,model)